In [1]:
import os
import glob
import openslide
from histoplus.extract import extract
from histoplus.helpers.segmentor import CellViTSegmentor
from histoplus.helpers.tissue_detection import detect_tissue_on_wsi

import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image, ImageDraw, ImageFont
from typing import Dict, Tuple, List

import math
from collections import Counter
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator
import xml.etree.ElementTree as ET

# os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [2]:
MPP = 0.25  # If available, otherwise set to 0.5
INFERENCE_IMAGE_SIZE = 588 # default = 784

In [3]:
wsi_list = glob.glob("/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/*.svs")
wsi_list

['/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-001_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-002_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-003_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-004_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-005_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-006_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-007_05.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-008_06.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-009_06.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-010_06.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-011_06.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-012_07.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-013_07.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-014_07.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-015_07.svs',
 '/home/na

In [4]:
from tqdm import tqdm
output_dir = './output/KBSMC/'
os.makedirs(output_dir, exist_ok = True)

for  wsi_path  in tqdm(wsi_list):
    
    file_name = wsi_path.split('/')[-1].split('.')[0].replace("-",'_')
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    if not os.path.exists(wsi_path):
        continue
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 32, # 786 -> 32
        ) 

        #case1 DB-000508-A1: vram 22000mib*3 사용 ram 30g 사용 20min
        #case2 1-028-08: vram 20000mib*3 사용 ram 10~30g 사용 20min / inference 8min[03:21<05:20,  1.73s/it] + 그 외 작업 20분 + filetering
        ### Filter overlapping cells (find-union):  63%|██████▎   | 403843/643120 [00:21<00:11, 20702.95it/s] -> (grouping) 1분? 
        ### Assigning cells to original tiles:  16%|█▌        | 3137/19670 [00:06<00:32, 502.39it/s] 1.5분정도

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

  0%|          | 0/222 [00:00<?, ?it/s]2025-12-04 15:35:13.070 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-12-04 15:35:13.071 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-12-04 15:35:15.772 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6898 tiles with Otsu's threshold.
2025-12-04 15:35:17.919 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  0%|          | 1/222 [18:04<66:34:17, 1084.42s/it]

OK → ./output/KBSMC//seg_1_001_04_588.json


2025-12-04 15:53:17.473 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-12-04 15:53:17.475 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-12-04 15:53:19.584 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 4613 tiles with Otsu's threshold.
2025-12-04 15:53:21.337 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
  1%|          | 2/222 [31:53<57:06:01, 934.37s/it] 

OK → ./output/KBSMC//seg_1_002_04_588.json


2025-12-04 16:07:06.786 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-12-04 16:07:06.788 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-12-04 16:07:10.511 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6384 tiles with Otsu's threshold.
2025-12-04 16:07:12.234 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  1%|▏         | 3/222 [48:54<59:14:15, 973.77s/it]

...

OK → ./output/KBSMC//seg_2_092_588.json


2025-12-07 08:39:26.229 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-12-07 08:39:26.230 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-12-07 08:39:29.654 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 8918 tiles with Otsu's threshold.
2025-12-07 08:39:31.628 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
100%|█████████▉| 221/222 [65:19:15<15:52, 952.83s/it]

OK → ./output/KBSMC//seg_2_093_588.json


2025-12-07 08:54:28.562 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-12-07 08:54:28.564 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-12-07 08:54:31.690 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 8682 tiles with Otsu's threshold.
2025-12-07 08:54:33.521 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
100%|██████████| 222/222 [65:35:27<00:00, 1063.64s/it]

OK → ./output/KBSMC//seg_2_094_588.json


In [5]:
asgdafd

NameError: name 'asgdafd' is not defined

In [ ]:
file_name  = "DB-000014-E1"


json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
if os.path.exists(json_path):
    print(1)
wsi_path = f'/data/sjhong/wsi/PNU/{file_name}.svs'
# wsi_path = '/home/sjhong/DB-000037-B2.svs'
if not os.path.exists(wsi_path):
    print(3)

slide = openslide.OpenSlide(wsi_path)
slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]

1


((160092, 85103), '0.26392599999999999')

In [ ]:
import h5py

with h5py.File(f"/home/sjhong/features/PNU/Uni_full_patch/{file_name}.h5","r") as f:
    coords = f['coords'][:]
coords.shape

(15335, 2)

In [ ]:
import numpy as np
from loguru import logger
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from histoplus.helpers.tiling.optimal_mpp import get_tiling_slide_level


def _apply_otsu_threshold(
    slide: OpenSlide, mask_width: int, mask_height: int
) -> np.ndarray:
    thumbnail = slide.get_thumbnail((mask_width, mask_height))
    img_arr = np.array(rgb2gray(thumbnail))
    thresh = threshold_otsu(img_arr)
    return (img_arr > thresh).astype(np.uint8)

def detect_tissue_on_wsi(
    slide: OpenSlide,
    matter_threshold: float = 0.2,
    target_mpp: float = 8.0,
    base_mpp: float = 0.5,
    tile_size_at_base_mpp: int = 224,
    default_mpp_max: float = 0.23,
    mpp_rtol: float = 0.2,
) -> tuple[np.ndarray, int]:
    downsample_factor = target_mpp / base_mpp
    tile = int(tile_size_at_base_mpp / downsample_factor)  # e.g., 224 / 16 = 14

    deepzoom = DeepZoomGenerator(slide, tile_size=tile_size_at_base_mpp, overlap=0)

    dz_level_at_target_mpp = get_tiling_slide_level(
        slide, deepzoom, mpp=target_mpp,
        default_mpp_max=default_mpp_max, mpp_rtol=mpp_rtol
    )
    dz_level_at_base_mpp = get_tiling_slide_level(
        slide, deepzoom, mpp=base_mpp,
        default_mpp_max=default_mpp_max, mpp_rtol=mpp_rtol
    )

    # DeepZoom의 목표 크기를 그대로 신뢰하지 말고, 마스크 실제 크기를 기준으로 계산
    width_t, height_t = deepzoom.level_dimensions[dz_level_at_target_mpp]

    mask = _apply_otsu_threshold(slide, width_t, height_t)  # shape: (H, W)
    H, W = mask.shape  # 반드시 (height, width) 확인

    # 마스크 실제 크기로 rows/cols 계산 (정수 나눗셈)
    rows = H // tile
    cols = W // tile

    # 크롭을 마스크 실제 크기에 맞춰 수행 (반드시 tile의 배수로 자르기)
    Hc = rows * tile
    Wc = cols * tile
    mask_cropped = mask[:Hc, :Wc]

    # 안전 체크: 요소 수 일치 확인
    assert mask_cropped.size == rows * cols * tile * tile, \
        f"size mismatch: {mask_cropped.size} vs {rows * cols * tile * tile}"

    # (rows, tile, cols, tile)로 리쉐이프 후 (1,3) 축 평균
    tiles = mask_cropped.reshape(rows, tile, cols, tile)
    tissue_scores = tiles.mean(axis=(1, 3))  # shape: (rows, cols)

    tissue_coords = np.array(np.nonzero(tissue_scores < matter_threshold)).T.astype(int)
    # Openslide 관례에 맞게 (row,col)->(x,y) 스왑
    tissue_coords[:, [0, 1]] = tissue_coords[:, [1, 0]]

    logger.info(f"Found {len(tissue_coords)} tiles with Otsu's threshold.")
    return tissue_coords, dz_level_at_base_mpp


In [ ]:
tissue_coords, dz_level = detect_tissue_on_wsi(slide)

2025-09-25 09:03:43.818 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.445632 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.263926.
2025-09-25 09:03:43.820 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.527852 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.263926.
2025-09-25 09:03:50.873 | INFO     | __main__:detect_tissue_on_wsi:68 - Found 5525 tiles with Otsu's threshold.


In [ ]:
def detect_tissue_on_wsi(
    slide: OpenSlide,
    matter_threshold: float = 0.2,
    target_mpp: float = 8.0,
    base_mpp: float = 0.5,
    tile_size_at_base_mpp: int = 224,
    default_mpp_max: float = 0.23,
    mpp_rtol: float = 0.2,
) -> tuple[np.ndarray, int]:
    """Detect tissue on WSI using Otsu's threshold."""
    # --- 기본 파라미터/타일 크기 계산 ---
    downsample_factor = target_mpp / base_mpp
    tile = int(tile_size_at_base_mpp / downsample_factor)  # 타겟 mpp에서의 타일 픽셀 수

    deepzoom = DeepZoomGenerator(slide, tile_size=tile_size_at_base_mpp, overlap=0)

    dz_level_at_target_mpp = get_tiling_slide_level(
        slide,
        deepzoom,
        mpp=target_mpp,
        default_mpp_max=default_mpp_max,
        mpp_rtol=mpp_rtol,
    )
    dz_level_at_base_mpp = get_tiling_slide_level(
        slide,
        deepzoom,
        mpp=base_mpp,
        default_mpp_max=default_mpp_max,
        mpp_rtol=mpp_rtol,
    )

    width_t, height_t = deepzoom.level_dimensions[dz_level_at_target_mpp]

    # --- 마스크 생성 (height, width 순서 확인) ---
    mask = _apply_otsu_threshold(slide, width_t, height_t)  # shape: (H, W)
    H, W = mask.shape

    # --- rows/cols은 마스크 실제 크기로 계산 ---
    rows = H // tile
    cols = W // tile

    Hc = rows * tile
    Wc = cols * tile
    mask_cropped = mask[:Hc, :Wc]

    # 🔍 디버깅 로깅 (리쉐이프 직전: 가장 유용한 지점)
    logger.debug({
        "dz_level_at_target_mpp": dz_level_at_target_mpp,
        "dz_level_at_base_mpp": dz_level_at_base_mpp,
        "deepzoom_level_dims(target)": (width_t, height_t),
        "mask_shape": (int(H), int(W)),
        "tile": int(tile),
        "rows_cols": (int(rows), int(cols)),
        "Hc_Wc": (int(Hc), int(Wc)),
        "expected_elems": int(rows * cols * tile * tile),
        "actual_elems": int(mask_cropped.size),
    })

    # --- 안전 체크: 요소 수 일치 보장 ---
    assert mask_cropped.size == rows * cols * tile * tile, \
        f"size mismatch: {mask_cropped.size} vs {rows * cols * tile * tile} (check tile/resize logic)"

    # --- 타일 단위 평균으로 tissue score 계산 ---
    tiles = mask_cropped.reshape(rows, tile, cols, tile)
    tissue_scores = tiles.mean(axis=(1, 3))  # shape: (rows, cols)

    # --- 스코어 기준 좌표 추출 ---
    tissue_coords = np.array(np.nonzero(tissue_scores < matter_threshold)).T.astype(int)

    # Openslide 관례 맞춰 (row, col) -> (x, y) 스왑
    if tissue_coords.size > 0:
        tissue_coords[:, [0, 1]] = tissue_coords[:, [1, 0]]

    logger.info(f"Found {len(tissue_coords)} tiles with Otsu's threshold.")

    return tissue_coords, dz_level_at_base_mpp


In [ ]:
tissue_coords, dz_level = detect_tissue_on_wsi(slide)

2025-09-25 09:18:30.625 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.445632 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.263926.
2025-09-25 09:18:30.627 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.527852 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.263926.
2025-09-25 09:18:39.039 | DEBUG    | __main__:detect_tissue_on_wsi:47 - {'dz_level_at_target_mpp': 13, 'dz_level_at_base_mpp': 17, 'deepzoom_level_dims(target)': (5003, 2660), 'mask_shape': (2659, 5003), 'tile': 14, 'rows_cols': (189, 357), 'Hc_Wc': (2646, 4998), 'expected_elems': 13224708, 'actual_elems': 13224708}
2025-09-25 09:18:39.070 | INFO     | __main__:detect_tissue_on_wsi:74 - Found 5525 tiles with Otsu's threshold.


In [ ]:
try:
    # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
    segmentor = CellViTSegmentor.from_histoplus(
        mpp=MPP,
        mixed_precision=True,                 # GPU AMP 사용 권장
        inference_image_size=INFERENCE_IMAGE_SIZE,
    )


except Exception as e:
    print("extract", e)

In [ ]:
# Process a whole slide image
results = extract(
    slide=slide,
    coords=tissue_coords,
    deepzoom_level=dz_level,
    segmentor=segmentor,
    batch_size= 8, # 786 -> 32
) 

results.save(json_path)
print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec


2025-09-25 09:04:38.783 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.263926 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.263926.


OK → ./output/pnu_0924//seg_DB-000014-E1_588.json


In [ ]:
13312/(60*60)

3.6977777777777776

In [ ]:

def detect_tissue_on_wsi(
    slide: OpenSlide,
    matter_threshold: float = 0.2,
    target_mpp: float = 8.0,
    base_mpp: float = 0.5,
    tile_size_at_base_mpp: int = 224,
    default_mpp_max: float = 0.23,
    mpp_rtol: float = 0.2,
) -> tuple[np.ndarray, int]:
    """Detect tissue on WSI using Otsu's thresholding."""
    downsample_factor = target_mpp / base_mpp
    tile = int(tile_size_at_base_mpp / downsample_factor)
    tile_size_at_target_mpp = int(tile_size_at_base_mpp / downsample_factor)

    deepzoom = DeepZoomGenerator(slide, tile_size=tile_size_at_base_mpp, overlap=0)

    dz_level_at_target_mpp = get_tiling_slide_level(
        slide,
        deepzoom,
        mpp=target_mpp,
        default_mpp_max=default_mpp_max,
        mpp_rtol=mpp_rtol,
    )

    dz_level_at_base_mpp = get_tiling_slide_level(
        slide,
        deepzoom,
        mpp=base_mpp,
        default_mpp_max=default_mpp_max,
        mpp_rtol=mpp_rtol,
    )

    width_at_target_mpp, height_at_target_mpp = deepzoom.level_dimensions[
        dz_level_at_target_mpp
    ]

    mask = _apply_otsu_threshold(slide, width_at_target_mpp, height_at_target_mpp)

    num_cols = math.floor(width_at_target_mpp / tile_size_at_target_mpp)
    num_rows = math.floor(height_at_target_mpp / tile_size_at_target_mpp)

    # Crop mask to make it reshapable (fix the math.floor)
    mask_cropped = mask[
        : num_rows * tile_size_at_target_mpp, : num_cols * tile_size_at_target_mpp
    ]
    
    H, W = mask.shape

    # --- rows/cols은 마스크 실제 크기로 계산 ---
    rows = H // tile
    cols = W // tile

    Hc = rows * tile
    Wc = cols * tile
    

    width_t, height_t = deepzoom.level_dimensions[dz_level_at_target_mpp]

    # 🔍 디버깅 로깅 (리쉐이프 직전: 가장 유용한 지점)
    logger.debug({
        "dz_level_at_target_mpp": dz_level_at_target_mpp,
        "dz_level_at_base_mpp": dz_level_at_base_mpp,
        "deepzoom_level_dims(target)": (width_t, height_t),
        "mask_shape": (int(H), int(W)),
        "tile": int(tile),
        "rows_cols": (int(rows), int(cols)),
        "Hc_Wc": (int(Hc), int(Wc)),
        "expected_elems": int(rows * cols * tile * tile),
        "actual_elems": int(mask_cropped.size),
    })
    tiles = mask_cropped.reshape(
        num_rows, tile_size_at_target_mpp, num_cols, tile_size_at_target_mpp
    )
    tissue_scores = tiles.mean(axis=(1, 3))

    tissue_coords = np.array(np.nonzero(tissue_scores < matter_threshold))
    tissue_coords = np.transpose(tissue_coords).astype(int)

    # Swapping columns to respect Openslide's conventions
    tissue_coords[:, [0, 1]] = tissue_coords[:, [1, 0]]

    logger.info(f"Found {len(tissue_coords)} tiles with Otsu's threshold.")

    return tissue_coords, dz_level_at_base_mpp


In [ ]:
tissue_coords, dz_level = detect_tissue_on_wsi(slide)

2025-09-25 09:33:16.955 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.445632 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.263926.
2025-09-25 09:33:16.956 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.527852 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.263926.
2025-09-25 09:33:23.940 | DEBUG    | __main__:detect_tissue_on_wsi:60 - {'dz_level_at_target_mpp': 13, 'dz_level_at_base_mpp': 17, 'deepzoom_level_dims(target)': (5003, 2660), 'mask_shape': (2659, 5003), 'tile': 14, 'rows_cols': (189, 357), 'Hc_Wc': (2646, 4998), 'expected_elems': 13224708, 'actual_elems': 13289682}


ValueError: cannot reshape array of size 13289682 into shape (190,14,357,14)

In [ ]:
tissue_coords, dz_level = detect_tissue_on_wsi(slide)

2025-09-25 09:26:49.856 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.445632 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.263926.
2025-09-25 09:26:49.857 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.527852 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.263926.
2025-09-25 09:26:57.438 | DEBUG    | __main__:detect_tissue_on_wsi:45 - {'mask_shape': (2659, 5003), 'tile_size_at_target_mpp': 14, 'num_rows': 190, 'num_cols': 357, 'expected_elems': 13294680, 'actual_elems': 13289682}


ValueError: cannot reshape array of size 13289682 into shape (190,14,357,14)

{'dz_level_at_target_mpp': 13, 'dz_level_at_base_mpp': 17, 'deepzoom_level_dims(target)': (5003, 2660), 'mask_shape': (2659, 5003), 'tile': 14, 'rows_cols': (189, 357), 'Hc_Wc': (2646, 4998), 'expected_elems': 13224708, 'actual_elems': 13224708}

{'dz_level_at_target_mpp': 13, 'dz_level_at_base_mpp': 17, 'deepzoom_level_dims(target)': (5003, 2660), 'mask_shape': (2659, 5003), 'tile': 14, 'rows_cols': (189, 357), 'Hc_Wc': (2646, 4998), 'expected_elems': 13224708, 'actual_elems': 13289682}

{'dz_level_at_target_mpp': 13, 'dz_level_at_base_mpp': 17, 'deepzoom_level_dims(target)': (5003, 2660), 'mask_shape': (2659, 5003), 'tile': 14, 'rows_cols': (189, 357), 'Hc_Wc': (2646, 4998), 'expected_elems': 13224708, 'actual_elems': 13289682}
{'dz_level_at_target_mpp': 13, 'dz_level_at_base_mpp': 17, 'deepzoom_level_dims(target)': (5003, 2660), 'mask_shape': (2659, 5003), 'tile': 14, 'rows_cols': (189, 357), 'Hc_Wc': (2646, 4998), 'expected_elems': 13224708, 'actual_elems': 13224708}

In [ ]:
slide.properties[openslide.PROPERTY_NAME_MPP_X], slide.properties[openslide.PROPERTY_NAME_MPP_Y]

('0.26392599999999999', '0.26392599999999999')

In [ ]:

for  file_name  in sorted(all_ids):
    
    wsi_path = f'/data/sjhong/wsi/PNU/{file_name}.svs'
    slide = openslide.OpenSlide(wsi_path)
    print(slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X], slide.properties[openslide.PROPERTY_NAME_MPP_Y])
    slide.close()

(160092, 85103) 0.26392599999999999 0.26392599999999999
(104837, 87866) 0.26392599999999999 0.26392599999999999
(154764, 77604) 0.26392599999999999 0.26392599999999999
(146279, 62804) 0.26392599999999999 0.26392599999999999
(146279, 62804) 0.26392599999999999 0.26392599999999999
(185747, 73657) 0.26392599999999999 0.26392599999999999
(128913, 87669) 0.26392599999999999 0.26392599999999999
(108784, 87866) 0.26392599999999999 0.26392599999999999
(108784, 87866) 0.26392599999999999 0.26392599999999999
(188115, 79578) 0.26392599999999999 0.26392599999999999
(188115, 79578) 0.26392599999999999 0.26392599999999999
(167789, 76617) 0.26392599999999999 0.26392599999999999
(170946, 83327) 0.26392599999999999 0.26392599999999999
(124769, 85300) 0.26392599999999999 0.26392599999999999
(111547, 71092) 0.26392599999999999 0.26392599999999999
(127531, 87866) 0.26392599999999999 0.26392599999999999
(109771, 83524) 0.26392599999999999 0.26392599999999999
(118256, 87866) 0.26392599999999999 0.2639259999